# Exercise on Genome Scale Metabolic Models - LIFE752 - cycle 2 part 1 (2)


First we need to import the cobra library

In [25]:
import cobra

load the e. coli model provided.

In [26]:
model = cobra.io.read_sbml_model('e_coli_core_life752.xml')


In [27]:
model

Name,e_coli_core
Memory address,15070af40
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


printing of the current constraints on the exchange reactions showing which metabolites are allowed to take up


In [28]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 15.0,
 'EX_glu__L_e': 2.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Running of FBA, to predict max growth rate



In [29]:
model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,15,6,90.00%
glu__L_e,EX_glu__L_e,2,5,10.00%
nh4_e,EX_nh4_e,6.014,0,0.00%
o2_e,EX_o2_e,34.76,0,0.00%
pi_e,EX_pi_e,5.407,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-37.45,1,100.00%
h2o_e,EX_h2o_e,-44.16,0,0.00%
h_e,EX_h_e,-25.48,0,0.00%


From the model.summary() output we can easily see what the maximum growth rate is.

In [30]:
model.reactions.BIOMASS_Ecoli_core_w_GAM.flux #max growth rate

1.4697423584066123

Changing to anaerobic conditions: the exchange reaction for oxygen (set lower bound to 0)

In [31]:
model.reactions.EX_o2_e.lower_bound=0

And next simply run FBA again

In [32]:
model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,15,6,90.00%
glu__L_e,EX_glu__L_e,2,5,10.00%
h2o_e,EX_h2o_e,12.21,0,0.00%
nh4_e,EX_nh4_e,0.08325,0,0.00%
pi_e,EX_pi_e,1.405,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
ac_e,EX_ac_e,-11.54,2,27.55%
co2_e,EX_co2_e,-0.9052,1,1.08%
etoh_e,EX_etoh_e,-13.48,2,32.20%
for_e,EX_for_e,-26.45,1,31.58%




To revert to aerobic condition, we bring the lower bound of the exchange reaction for o2 to -1000

In [33]:
model.reactions.EX_o2_e.lower_bound=-1000

And then run the FVA analysis with the code below

In [34]:
res = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.9) # fraction_of_optimum set to 0.9

In [35]:
res.loc['EX_etoh_e'] # max ethanol


minimum    0.000
maximum    3.724
Name: EX_etoh_e, dtype: float64

#Next block reloads the model, using pfba to find max growth. Model is constrained to grow at least at 85% of max growth, with acetate export set as objective function

In [36]:
import cobra
from cobra.flux_analysis import pfba

# Reloading of the model to reset all previous changes
model = cobra.io.read_sbml_model('e_coli_core_life752.xml')

# pFBA ran to get the maximum growth rate
pfba_solution = pfba(model)
max_growth = pfba_solution.objective_value
print("Maximum growth rate (pFBA):", max_growth)

# constrained biomass reaction to ≥85% of max
biomass_rxn = model.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
biomass_rxn.lower_bound = 0.85 * max_growth

# acetate export set as the new objective
model.objective = 'EX_ac_e'

# set to maximize acetate production
acetate_solution = model.optimize()
print("Maximum acetate production rate (EX_ac_e):", acetate_solution.objective_value)


Maximum growth rate (pFBA): 798.8666866804758
Maximum acetate production rate (EX_ac_e): 1000.0


/Users/alexbeggs/miniforge3/envs/life752_env/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
